In [3]:
import sqlite3
import pandas as pd
import numpy as np



# NOTA
El DataFrame df se crea a partir de la tabla basa_datos_pripal de la base de datos credit_scoring.db. El archivo no se incluye en el repositorio debido a su tamaño; la generación de la base de datos a partir del CSV original y la creación de la tabla se explica detalladamente en el notebook data-collection.ipynb

Descriptive analysis:
In this part of the project, we begin exploring the dataset created from the initial information obtained from the LendingClub dataset (Kaggle). The objective of this stage is to describe and understand the structure of the data, the variables and their types, their distributions, skewness, and the presence of missing values.

In [ ]:
# pd.read_sql(
#     "SELECT name FROM sqlite_master WHERE type='table';",
#     conn
# )

: 

In [2]:
conn = sqlite3.connect("/workspaces/final_project_creditscoring/Data/credit_scoring.db")
df = pd.read_sql("SELECT * FROM basa_datos_pripal", conn)
conn.close()



In [ ]:
# conn = sqlite3.connect("/workspaces/final_project_creditscoring/Data/credit_scoring.db")
# df = pd.read_sql("SELECT * FROM basa_datos_pripal", conn)
# conn.close()
# n_rows,n_cols = df.shape

# print(f'En este df existen {n_rows} filas y {n_cols} columnas')

: 

In [4]:
conn = sqlite3.connect("/workspaces/final_project_creditscoring/Data/credit_scoring.db")

df = pd.read_sql(
    """
    SELECT *
    FROM basa_datos_pripal
    ORDER BY RANDOM()
    LIMIT 30000
    """,
    conn
)

conn.close()
df.shape


(30000, 151)

In [5]:
n_rows,n_cols = df.shape
print(f'En este df existen {n_rows} filas y {n_cols} columnas')

En este df existen 30000 filas y 151 columnas


With the analysis below, we can understand that the dataset contains a large number of numerical variables, along with several categorical features represented as object types. This initial inspection highlights the need for feature selection and type handling in later stages. Now we are proceeding with a list of each type of column to identify possible issues in data types (such as date columns that are objects or numerical data that is shown as objects, etc.).

In [ ]:
cols_types = df.dtypes.reset_index().rename(
    columns={'index': 'column_name', 0: 'dtype'}
)

cols_types['dtype'].value_counts()

dtype
float64    112
object      38
int64        1
Name: count, dtype: int64

In [ ]:
cols_types[cols_types['dtype'] == 'object']

,column_name,dtype
1,member_id,object
5,term,object
8,grade,object
9,sub_grade,object
10,emp_title,object
11,emp_length,object
12,home_ownership,object
14,verification_status,object
15,issue_d,object
16,loan_status,object


In [ ]:
print(cols_types[cols_types['dtype'] == 'float64'].to_string(index=False))

                               column_name   dtype
                                 loan_amnt float64
                               funded_amnt float64
                           funded_amnt_inv float64
                                  int_rate float64
                               installment float64
                                annual_inc float64
                                       dti float64
                               delinq_2yrs float64
                            fico_range_low float64
                           fico_range_high float64
                            inq_last_6mths float64
                    mths_since_last_delinq float64
                    mths_since_last_record float64
                                  open_acc float64
                                   pub_rec float64
                                 revol_bal float64
                                revol_util float64
                                 total_acc float64
                               

In [ ]:
cols_types[cols_types['dtype'] == 'int64']

,column_name,dtype
0,id,int64


Revision of constant columns: 

The following code initially didn’t specify dropna=False, which made it show a few columns as constants. This led us to investigate what was happening and whether we were working with the right dataframe. However, this mistake was enlightening, as it helped us identify possible *data leakage variables, such as: hardship_type, deferral_term, and hardship_length.*

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

# Unique values per column
uniq = df.nunique(dropna=False)

# Show results
print(uniq)


id                                            192309
member_id                                          1
loan_amnt                                       1447
funded_amnt                                     1447
funded_amnt_inv                                 4951
term                                               2
int_rate                                         499
installment                                    42629
grade                                              7
sub_grade                                         35
emp_title                                      92975
emp_length                                        12
home_ownership                                     6
annual_inc                                     15656
verification_status                                3
issue_d                                          103
loan_status                                        9
pymnt_plan                                         2
url                                           

In [ ]:
uniq = df.nunique(dropna=False)
uniq[uniq == 1]

member_id      1
policy_code    1
dtype: int64

After the code revision, the only two variables with constant values are member_id and policy_code. It does not make much sense to have a unique value for member_id if we have almost 200k data entries, so we needed to check the exact values contained in this column.

In [ ]:
cols = ['member_id', 'policy_code']

for col in cols:
    print(df[col].value_counts(dropna=False))

member_id
None    192309
Name: count, dtype: int64
policy_code
1.0    192309
Name: count, dtype: int64


After doing so, we discovered that it only contains None, which means the column is not relevant at all for this project and can therefore be discarded.

Regarding policy_code, a similar situation occurs. According to the data dictionary, LendingClub has only two types of policies: publicly available (1) and new products not publicly available (2). In this dataset, only publicly available products are present. Therefore, and following the same reasoning as above, policy_code is not a relevant column for the analysis.

In [ ]:
df = df.drop(columns=['member_id', 'policy_code'])

Revision of Duplicated Rows: No duplicated rows were identified.

In [ ]:
df.duplicated().sum()

np.int64(0)

Revision of Duplicated Columns:  

In [ ]:
df.T.duplicated().sum()
df.T.duplicated(keep=False)

id                                            False
loan_amnt                                     False
funded_amnt                                   False
funded_amnt_inv                               False
term                                          False
int_rate                                      False
installment                                   False
grade                                         False
sub_grade                                     False
emp_title                                     False
emp_length                                    False
home_ownership                                False
annual_inc                                    False
verification_status                           False
issue_d                                       False
loan_status                                   False
pymnt_plan                                    False
url                                           False
desc                                          False
purpose     

Two variables (deferral_term and hardship_length) were found to be exact duplicates, containing identical values across all observations. Both variables are related to post-loan hardship events (we previously identified them as potential data leakers) and will therefore be excluded from the modeling stage. 

Identify Missing Values

In [ ]:
missing = df.isna().mean()*100
missing[missing>0]
print(missing)

id                                             0.000000
loan_amnt                                      0.000000
funded_amnt                                    0.000000
funded_amnt_inv                                0.000000
term                                           0.000000
int_rate                                       0.000000
installment                                    0.000000
grade                                          0.000000
sub_grade                                      0.000000
emp_title                                      6.813514
emp_length                                     5.422523
home_ownership                                 0.000000
annual_inc                                     0.000000
verification_status                            0.000000
issue_d                                        0.000000
loan_status                                    0.000000
pymnt_plan                                     0.000000
url                                            0

We identified columns with a high percentage of missing values, so we proceeded to define a missing threshold of 50%, where variables with more than 50% missing values will be considered for exclusion from the modeling stage. However, first we must evaluate them on a case-by-case basis to understand if any of those variables are conceptually important.

In [ ]:
missing_threshold = 50

high_missing_cols = missing[missing >= missing_threshold]
print(high_missing_cols)

desc                                          77.978670
mths_since_last_delinq                        54.782147
mths_since_last_record                        87.175327
next_pymnt_d                                  75.705245
mths_since_last_major_derog                   80.238054
annual_inc_joint                              96.907581
dti_joint                                     96.907581
verification_status_joint                     97.014180
open_acc_6m                                   63.197250
open_act_il                                   63.197250
open_il_12m                                   63.197250
open_il_24m                                   63.197250
mths_since_rcnt_il                            64.383882
total_bal_il                                  63.197250
il_util                                       68.575574
open_rv_12m                                   63.197250
open_rv_24m                                   63.197250
max_bal_bc                                    63

In [ ]:
(missing > 50).sum()

np.int64(57)

Columns with more than 50% missing values were manually reviewed and classified into post-loan variables, second-applicant features, structurally missing variables, and late-reported behavioral features based on domain knowledge and data documentation. No features were removed at this stage; the analysis documents decisions to be applied during model preparation.

In [ ]:
high_missing_review = pd.DataFrame({
    "column": high_missing_cols.index,
    "missing_pct": high_missing_cols.values
})

In [ ]:
def classify_column(col):
    if col.startswith("hardship") or col.startswith("settlement") or col.startswith("deferral"):
        return "post_loan"
    if col.startswith("sec_app") or col.endswith("_joint"):
        return "second_applicant"
    if col.startswith("mths_since"):
        return "structural_missing"
    return "late_reported"

In [ ]:
high_missing_review["category"] = high_missing_review["column"].apply(classify_column)

high_missing_review

,column,missing_pct,category
0,desc,77.978670,late_reported
1,mths_since_last_delinq,54.782147,structural_missing
2,mths_since_last_record,87.175327,structural_missing
3,next_pymnt_d,75.705245,late_reported
4,mths_since_last_major_derog,80.238054,structural_missing
5,annual_inc_joint,96.907581,second_applicant
6,dti_joint,96.907581,second_applicant
7,verification_status_joint,97.014180,second_applicant
8,open_acc_6m,63.197250,late_reported
9,open_act_il,63.197250,late_reported


1. *Post-loan variables:*

These features contain information generated after loan origination, such as hardship or settlement events. Their high missingness reflects the fact that most loans do not enter these processes. Because these variables include future information relative to the credit decision, they were identified as potential sources of data leakage.

Planned decision: Exclude during model preparation.

2. *Second-applicant variables:*

These variables describe characteristics of a co-borrower in joint loan applications. The high proportion of missing values reflects that most loans involve a single applicant, meaning missing values indicate the absence of a second applicant rather than missing information.

Rather than modeling the full co-borrower profile, the presence of a second applicant is captured through a binary indicator. This approach preserves potentially relevant information while avoiding additional complexity and extensive imputation.

Planned decision: Create a binary flag indicating whether a loan includes a second applicant, and exclude detailed second-applicant features during model preparation.

3. *Structurally missing variables:*

These features represent the time since the last occurrence of negative credit events. Missing values indicate that the event has never occurred, making the missingness itself informative.

Planned decision: Retain for modeling and apply a dedicated imputation strategy at a later stage.

4. *Late-reported features:*

These variables were introduced into the dataset at later periods and are unavailable for older loans. Missingness is driven by historical reporting limitations rather than borrower behavior.

Planned decision: Evaluate after defining the temporal train-test split.

In [ ]:
zero_ratio = (df == 0).mean() * 100
zero_ratio = zero_ratio[zero_ratio > 0].sort_values(ascending=False)

zero_ratio.head(20)

delinq_amnt                   99.771202
acc_now_delinq                99.702042
chargeoff_within_12_mths      99.412924
collections_12_mths_ex_med    98.901247
tax_liens                     97.929374
total_rec_late_fee            96.115106
collection_recovery_fee       90.665023
pub_rec_bankruptcies          90.167387
recoveries                    89.107634
pub_rec                       87.175327
delinq_2yrs                   83.199954
num_tl_30dpd                  76.676079
out_prncp                     76.144122
out_prncp_inv                 76.144122
num_tl_120dpd_2m              74.385494
num_tl_90g_dpd_24m            72.702786
tot_coll_amt                  65.989631
num_accts_ever_120_pd         59.765274
inq_last_6mths                56.788814
mort_acc                      34.524645
dtype: float64

In [ ]:
#Identify other missing values
cat_col = df.select_dtypes(include=['object']).columns

for col in cat_col: 
    print(df[col].value_counts())

term
36 months    137472
60 months     54837
Name: count, dtype: int64
grade
B    57542
C    51332
A    37335
D    27757
E    12718
Name: count, dtype: int64
sub_grade
B3    12706
B4    12370
C1    12100
B5    11572
C2    11040
Name: count, dtype: int64
emp_title
None          13103
Teacher        2338
Manager        1952
Owner          1249
Supervisor      933
Name: count, dtype: int64
emp_length
10+ years    60263
2 years      17674
< 1 year     16144
3 years      15639
5 years      13163
Name: count, dtype: int64
home_ownership
MORTGAGE    93952
RENT        79309
OWN         18968
ANY            52
OTHER          15
Name: count, dtype: int64
verification_status
Source Verified    67530
Not Verified       62455
Verified           62324
Name: count, dtype: int64
issue_d
2011-10-01 00:00:00    1941
2011-11-01 00:00:00    1941
2011-12-01 00:00:00    1941
2012-01-01 00:00:00    1941
2012-02-01 00:00:00    1941
Name: count, dtype: int64
loan_status
Fully Paid            119522
Current    

In [ ]:
df_faltantes = df.replace(['None'],np.nan,inplace=True)
df

Aca me quede. Queda pendiente de todas las columnas que tienen mas de 50%, 
1. analizar por que tienen valores faltantes y si realmente debemos eliminarnas
2. Contar el total de columnas que tienen mas de 50% de valores faltantes. 
3. Con cuales nos vamos a quedar para imputar?

In [ ]:
low_variance_cols = []

for col in df.columns:
    vc = df[col].value_counts(dropna=False, normalize=True)
    if vc.iloc[0] > 0.99:   # más del 99% el mismo valor
        low_variance_cols.append(col)

low_variance_cols

['pymnt_plan',
 'acc_now_delinq',
 'chargeoff_within_12_mths',
 'delinq_amnt',
 'sec_app_mths_since_last_major_derog',
 'hardship_flag',
 'hardship_type',
 'hardship_reason',
 'hardship_status',
 'deferral_term',
 'hardship_amount',
 'hardship_start_date',
 'hardship_end_date',
 'payment_plan_start_date',
 'hardship_length',
 'hardship_dpd',
 'hardship_loan_status',
 'orig_projected_additional_accrued_interest',
 'hardship_payoff_balance_amount',
 'hardship_last_payment_amount']

In [ ]:
desc = df.describe().T
desc.sort_values(by='max', ascending=False).head(20)

,count,mean,std,min,25%,50%,75%,max
id,192309.0,4.987646e+07,5.014201e+07,66128.0,1596224.00,3.435340e+07,9.417362e+07,1.456375e+08
annual_inc,192309.0,7.516959e+04,6.923942e+04,0.0,45000.00,6.400000e+04,9.000000e+04,8.700000e+06
tot_hi_cred_lim,147812.0,1.744076e+05,1.751981e+05,0.0,49067.75,1.120985e+05,2.542230e+05,5.850873e+06
tot_cur_bal,147812.0,1.405798e+05,1.571303e+05,0.0,28571.75,7.856900e+04,2.119948e+05,5.327039e+06
annual_inc_joint,5947.0,1.227523e+05,6.579404e+04,13464.0,82772.00,1.100000e+05,1.460000e+05,1.500000e+06
total_rev_hi_lim,147812.0,3.339641e+04,3.332634e+04,0.0,14400.00,2.470000e+04,4.150000e+04,1.417100e+06
total_bal_ex_mort,157627.0,4.885828e+04,4.721588e+04,0.0,20217.00,3.643100e+04,6.160950e+04,1.276247e+06
total_il_high_credit_limit,147812.0,4.176731e+04,4.333643e+04,0.0,14006.75,3.108400e+04,5.641425e+04,1.214546e+06
revol_bal,192309.0,1.604589e+04,2.050766e+04,0.0,5860.00,1.121500e+04,1.980900e+04,1.190046e+06
total_bal_il,70775.0,3.522341e+04,4.346104e+04,0.0,8607.50,2.289000e+04,4.601550e+04,8.017790e+05


The analysis of the maximum values reveals the presence of extreme values in some financial variables, which suggests the need to apply transformations or outlier treatment techniques in later stages.

In [ ]:
desc.assign(
    mean_median_ratio = desc['mean'] / desc['50%']
).sort_values('mean_median_ratio', ascending=False).head(10)

,count,mean,std,min,25%,50%,75%,max,mean_median_ratio
inq_last_6mths,192309.0,0.681424,0.977064,0.0,0.0,0.0,1.0,8.000000,inf
pub_rec,192309.0,0.157767,0.502036,0.0,0.0,0.0,0.0,40.000000,inf
delinq_2yrs,192309.0,0.269530,0.798124,0.0,0.0,0.0,0.0,26.000000,inf
collections_12_mths_ex_med,192309.0,0.012158,0.132566,0.0,0.0,0.0,0.0,20.000000,inf
total_rec_late_fee,192309.0,1.415083,11.190303,0.0,0.0,0.0,0.0,1098.360001,inf
recoveries,192309.0,153.452068,774.136498,0.0,0.0,0.0,0.0,35581.880000,inf
out_prncp_inv,192309.0,2404.686422,5862.737964,0.0,0.0,0.0,0.0,39091.640000,inf
open_il_12m,70775.0,0.670816,0.925402,0.0,0.0,0.0,1.0,20.000000,inf
acc_now_delinq,192309.0,0.003146,0.059249,0.0,0.0,0.0,0.0,3.000000,inf
tot_coll_amt,147812.0,201.409906,1907.319784,0.0,0.0,0.0,0.0,380757.000000,inf


Several numerical variables present highly skewed distributions, with median values equal to zero and a small proportion of non-zero observations. This pattern is expected for count-based credit history variables. However, some highly skewed variables correspond to post-loan information and will therefore be excluded from the modeling process to prevent data leakage.

EDA (cerrando)
1. Tratamiento columnas con alto % de 0s ⏳
2. Matriz de missing (missingno) ⏳
3. Valores faltantes explícitos ✅
4. Valores faltantes ocultos ✅
5. Filas duplicadas ✅
6. Drop policy_code ✅

Decisiones de features
7. Identificar columnas data leakage ⏳(ya estan identificadas, ahora hay que hacer drop)
8. Definir estrategia second applicant (flag + drop cols) ⏳
9. Identificar columnas ID / no predictivas ⏳(ya estan identificadas, ahora hay que hacer drop)
10. Definir target ⏳(ya identificado)

Modelado
11. Crear df_model
12. Split temporal
13. Imputación / preprocessing